# A 10-minute Tutorial on How to Use `HFTA`

This notebook demonstrates the way to integrate HFTA into a simple MNIST training example.

## Setup

Install the HFTA library from GitHub.

In [ ]:
!pip install git+https://github.com/UofT-EcoSystem/hfta

  Cloning https://github.com/UofT-EcoSystem/hfta to /tmp/pip-req-build-he46_x8d
  Running command git clone -q https://github.com/UofT-EcoSystem/hfta /tmp/pip-req-build-he46_x8d
  Created wheel for hfta: filename=hfta-0.1.0-cp37-none-any.whl size=74269 sha256=e2c6643cb5b33224b748792a156fac92ca7f97f366b0017944e6e116fe73b7d3
  Stored in directory: /tmp/pip-ephem-wheel-cache-8xzf8h_x/wheels/8d/02/01/48209526aba427578fbcd6b15919bac295ec9989de72f13ca6
Successfully built hfta


### Demo with a benchmark

Here is a demo run on one of the benchmarks provided in the `hfta` GitHub repo to make sure HFTA is properly installed.

Check [here](https://github.com/UofT-EcoSystem/hfta/tree/main/examples/mobilenet) for the code of this example (MobileNet V2).

In [ ]:
# We need to sync down the GitHub repo to run the benchmarks
!git clone https://github.com/UofT-EcoSystem/hfta

Cloning into 'hfta'...
remote: Enumerating objects: 1288, done.
remote: Counting objects: 100% (318/318), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 1288 (delta 210), reused 232 (delta 170), pack-reused 970
Receiving objects: 100% (1288/1288), 35.97 MiB | 32.03 MiB/s, done.
Resolving deltas: 100% (787/787), done.


In [ ]:
# Run the MobileNet V2 benchmark
!python hfta/examples/mobilenet/main.py --version v2 --epochs 5 --amp --eval --dataset cifar10 --device cuda --lr 0.01 0.02 0.03 --hfta

2021-04-27 06:04:15.968667: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Enable cuDNN heuristics!
170499072it [00:01, 89875879.80it/s]                   
Extracting /content/hfta/examples/mobilenet/../../datasets/cifar10/cifar-10-python.tar.gz to /content/hfta/examples/mobilenet/../../datasets/cifar10
Files already downloaded and verified
Epoch 0 took 27.782680988311768 s!
Epoch 1 took 15.300712585449219 s!
Epoch 2 took 15.644184350967407 s!
Epoch 3 took 15.658564567565918 s!
Epoch 4 took 15.461482524871826 s!
Running validation loop ...


Now, let's learn how to leverage HFTA on a normal PyTorch model in the following sections with a simple example of training a convolutional neural network on the MNIST dataset.

## Train a MNIST model without HFTA

We train a simple neural network with two convolutional layers and two fully connected layers, together with some max pooling and dropout layers. This model is trained with the MNIST dataset to recognize hand-written images.

### Define the model in the usual way

In [ ]:
import time
import random
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, 3, 1)
    self.conv2 = nn.Conv2d(32, 64, 3, 1)
    self.max_pool2d = nn.MaxPool2d(2)
    self.fc1 = nn.Linear(9216, 128)
    self.fc2 = nn.Linear(128, 10)
    self.dropout1 = nn.Dropout2d(0.25)
    self.dropout2 = nn.Dropout2d(0.5)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = self.max_pool2d(x)
    x = self.dropout1(x)
    x = torch.flatten(x, -3)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dropout2(x)
    x = self.fc2(x)
    output = F.log_softmax(x, dim=-1)
    return output

### Define the training and testing loop for a single epoch

In [ ]:
def train(config, model, device, train_loader, optimizer, epoch):
  """
  config: a dict defined by users to control the experiment
          See section: "Train the model"
  model: class Net defined in the code block above
  device: torch.device
  train_loader: torch.utils.data.dataloader.DataLoader
  optimizer: torch.optim
  epoch: int
  """
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % config["log_interval"] == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch,
          batch_idx * len(data),
          len(train_loader.dataset),
          100. * batch_idx / len(train_loader),
          loss.item(),
      ))
      if config["dry_run"]:
        break


def test(model, device, test_loader):
  """
  model: class Net defined in the code block above
  device: torch.device
  test_loader: torch.utils.data.dataloader.DataLoader
  """
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      N = target.size(0)
      output = model(data)
      test_loss += F.nll_loss(output, target,
                              reduction='none').view(-1, N).sum(dim=1)
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).view(-1, N).sum(dim=1)

  length = len(test_loader.dataset)
  test_loss /= length
  loss_str = ["%.4f" % e for e in test_loss]
  correct_str = [
      "%d/%d(%.2lf%%)" % (e, length, 100. * e / length) for e in correct
  ]
  print('Test set: \tAverage loss: {}, \n \t\t\tAccuracy: {}\n'.format(
      loss_str, correct_str))

### Define the main loop

In [ ]:
def main(config):
  """
  config: a dict defined by users to control the experiment
  """
  random.seed(1)
  np.random.seed(1)
  torch.manual_seed(1)

  device = torch.device(config["device"])

  kwargs = {'batch_size': config["batch_size"]}
  kwargs.update({'num_workers': 1, 'pin_memory': True, 'shuffle': True},)

  transform = transforms.Compose(
      [transforms.ToTensor(),
       transforms.Normalize((0.1307,), (0.3081,))])

  dataset1 = datasets.MNIST('./data',
                            train=True,
                            download=True,
                            transform=transform)
  dataset2 = datasets.MNIST('./data', train=False, transform=transform)
  train_loader = torch.utils.data.DataLoader(dataset1, **kwargs)
  test_loader = torch.utils.data.DataLoader(dataset2, **kwargs)

  model = Net().to(device)

  optimizer = optim.Adadelta(
      model.parameters(),
      lr=config["lr"][0],
  )

  start = time.perf_counter()
  for epoch in range(1, config["epochs"] + 1):
    now = time.perf_counter()
    train(config, model, device, train_loader, optimizer, epoch)
    print('Epoch {} took {} s!'.format(epoch, time.perf_counter() - now))
  end = time.perf_counter()

  test(model, device, test_loader)

  print('All jobs Finished, Each epoch took {} s on average!'.format(
      (end - start) / config["epochs"]))

### Train the model

In [ ]:
config = {
    "device": "cuda",  # choose from cuda and cpu
    "batch_size": 64,
    "lr": [1.0],
    "gamma": 0.7,
    "epochs": 4,
    "seed": 1,
    "log_interval": 500,
    "dry_run": False,
    "save_model": False,
}

print(config)
main(config)

{'device': 'cuda', 'batch_size': 64, 'lr': [1.0], 'gamma': 0.7, 'epochs': 4, 'seed': 1, 'log_interval': 500, 'dry_run': False, 'save_model': False}
Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.320749
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.066510
Epoch 1 took 11.140597851000024 s!
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.085425
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.029257
Epoch 2 took 11.133023817000009 s!
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.036951
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.170459
Epoch 3 took 11.134598162999993 s!
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.029309
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.040325
Epoch 4 took 11.170153673999977 s!
Test set: 	Average loss: ['0.0328'], 
 			Accuracy: ['9896/10000(98.96%)']

All jobs Finished, Each epoch took 11.144698025499999 s on average!


## Improve hardware utilization with HFTA

### How to modify a mnist model to use HFTA?

Our convolutional neural network was training fine with MNIST, and that's great! However, with such a small model and a small batch size, the underlying accelerator (NVIDIA GPU in this case) is likely going to be under-utilized. Thus, how can we possibly increase the hardware utilization for this training workload?

If this training workload is used under a repetitive setting (e.g., hyper-parameter tuning or convergence stability testing), hardware utilization can be directly increased by horizontally fusing multiple training workloads together, such that multiple models are trained on the same accelerator (e.g., GPU) at the same time.

However, fusing training workloads manually could be cumbersome and error-prone. Thus, the HFTA library provides convenient utilities to facilitate the effort of horizontally fusing models. Now, let us take a look into how we can easily perform the horizontal model fusion.

Please check the comments in the code to understand what needs to be done. In this example, we fuse multiple models (where the number of models is controlled by the parameter `B`) with different learning rates together via HFTA to improve the hardware utilization.

Be aware that this is just a very simple example of tuning the learning rate. However, in general, many other use cases might be applicable. For example: testing the convergence with different seeds; trying different weight initializers; or even ensemble learning.





#### Modify the model

In [ ]:
from __future__ import print_function
import sys
import time
import random
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Use utilities from the hfta package to convert your operators and optimizors.
from hfta.ops import convert_ops
from hfta.optim import get_hfta_optim_for


class Net(nn.Module):

  # When initializing the model, save the number of models that need to be fused 
  # (B), and convert the default operators to their HFTA version with 
  # convert_ops(B, list of operators).
  # When passing 0 to B, we train the model as it is without enabling HFTA.
  def __init__(self, B=0):
    super(Net, self).__init__()
    # Convert default operators to their HFTA version.
    (Conv2d, MaxPool2d, Linear, Dropout2d) = convert_ops(
        B,
        nn.Conv2d,
        nn.MaxPool2d,
        nn.Linear,
        nn.Dropout2d,
    )

    # Define the model with converted operators as if they were unchanged.
    self.B = B
    self.conv1 = Conv2d(1, 32, 3, 1)
    self.conv2 = Conv2d(32, 64, 3, 1)
    self.max_pool2d = MaxPool2d(2)
    self.fc1 = Linear(9216, 128)
    self.fc2 = Linear(128, 10)
    self.dropout1 = Dropout2d(0.25)
    self.dropout2 = Dropout2d(0.5)

  # Minor modifications to the forward pass on special operators.
  # Check the documentation of each operator for details.
  # Now the shape of x is [batch size, B, 3, 28, 28].
  # This means that the input images to all B models are concatenated along the 
  # channel dimension.
  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = self.max_pool2d(x)
    x = self.dropout1(x)

    x = torch.flatten(x, -3)
    if self.B > 0:
      # The output shape from flatten is [batch size, B, features], where
      # features == channels * height * width from dropout1.
      # However, fc1 expects the input shape to be [B, batch size, features].
      # Thus, we need to transpose the first and second dimensions.
      x = x.transpose(0, 1)

    x = self.fc1(x)
    x = F.relu(x)
    x = self.dropout2(x)
    x = self.fc2(x)
    output = F.log_softmax(x, dim=-1)
    return output

#### Modify the training and testing loop

In [ ]:
def train(config, model, device, train_loader, optimizer, epoch, B):
  """
  config: a dict defined by users to control the experiment
          See section: "Train the model"
  model: class Net defined in the code block above
  device: torch.device
  train_loader: torch.utils.data.dataloader.DataLoader
  optimizer: torch.optim
  epoch: int
  B: int, the number of models to be fused. When B == 0, we train the original 
     model as it is without enabling HFTA.
  """
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)

    # Need to duplicate a single batch of input images into multiple batches to 
    # feed into the fused model.
    if B > 0:
      N = target.size(0)
      data = data.unsqueeze(1).expand(-1, B, -1, -1, -1)
      target = target.repeat(B)

    optimizer.zero_grad()
    output = model(data)

    # Also need to modify the loss function to take consideration on the fused 
    # model.
    # In the case:
    #   1) the loss function is reduced by averaging along the batch dimension.
    #   2) multiple models are horizontally fused via HFTA.
    # To make sure the mathematically equivalent gradients are derived by 
    # ".backward()", we need to scale the loss value by B.
    # You might refer to our paper for why such scaling is needed.
    if B > 0:
      loss = B * F.nll_loss(output.view(B * N, -1), target)
    else:
      loss = F.nll_loss(output, target)

    loss.backward()
    optimizer.step()
    if batch_idx % config["log_interval"] == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))
      if config["dry_run"]:
        break


def test(model, device, test_loader, B):
  """
  model: class Net defined in the code block above
  device: torch.device
  test_loader: torch.utils.data.dataloader.DataLoader
  B: int, the number of models to be fused. When B == 0, we test the original 
     model as it is without enabling HFTA.
  """
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      N = target.size(0)

      # Need to duplicate a single batch of input images into multiple batches 
      # to feed into the fused model.
      if B > 0:
        data = data.unsqueeze(1).expand(-1, B, -1, -1, -1)
        target = target.repeat(B)

      output = model(data)

      # Change the shape of the output to align with the loss function.
      if B > 0:
        output = output.view(B * N, -1)

      test_loss += F.nll_loss(output, target,
                              reduction='none').view(-1, N).sum(dim=1)
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).view(-1, N).sum(dim=1)

  length = len(test_loader.dataset)
  test_loss /= length
  loss_str = ["%.4f" % e for e in test_loss]
  correct_str = [
      "%d/%d(%.2lf%%)" % (e, length, 100. * e / length) for e in correct
  ]
  print('Test set: \tAverage loss: {}, \n \t\t\tAccuracy: {}\n'.format(
      loss_str, correct_str))

#### Modify the main loop

In [ ]:
def main(config):
  """
  config: a dict defined by users to control the experiment
  """
  random.seed(config["seed"])
  np.random.seed(config["seed"])
  torch.manual_seed(config["seed"])

  device = torch.device(config["device"])

  kwargs = {'batch_size': config["batch_size"]}
  kwargs.update({'num_workers': 1, 'pin_memory': True, 'shuffle': True},)

  transform = transforms.Compose(
      [transforms.ToTensor(),
       transforms.Normalize((0.1307,), (0.3081,))])

  # Determine the number of models that are horizontally fused together from the 
  # number of provided learning rates that need to be tested.
  B = len(config["lr"]) if config["use_hfta"] else 0

  dataset1 = datasets.MNIST('./data',
                            train=True,
                            download=True,
                            transform=transform)
  dataset2 = datasets.MNIST('./data', train=False, transform=transform)
  train_loader = torch.utils.data.DataLoader(dataset1, **kwargs)
  test_loader = torch.utils.data.DataLoader(dataset2, **kwargs)

  # Specify the number of models that need to be fused horizontally together (B)
  # and create the fused model.
  model = Net(B).to(device)

  print('B={} lr={}'.format(B, config["lr"]), file=sys.stderr)

  # Convert the default optimizor (PyTorch Adadelta) to its HFTA version with 
  # get_hfta_optim_for(<default>, B).
  optimizer = get_hfta_optim_for(optim.Adadelta, B=B)(
      model.parameters(),
      lr=config["lr"] if B > 0 else config["lr"][0],
  )

  start = time.perf_counter()
  for epoch in range(1, config["epochs"] + 1):
    now = time.perf_counter()
    train(config, model, device, train_loader, optimizer, epoch, B)
    print('Epoch {} took {} s!'.format(epoch, time.perf_counter() - now))
  end = time.perf_counter()

  test(model, device, test_loader, B)

  print('All jobs Finished, Each epoch took {} s on average!'.format(
      (end - start) / (max(B, 1) * config["epochs"])))

### Train a single HFTA-enabled model with MNIST

Note that this run may be slightly slower than the non-HFTA version because enabling HFTA might lead to a small amount of overhead.

In [ ]:
# Enable HFTA to train only a single model.
# Only 1 model is trained
config = {
    "use_hfta": True,
    "device": "cuda",  # choose from cuda and cpu
    "batch_size": 64,
    "lr": [0.1],
    "gamma": 0.7,
    "epochs": 4,
    "seed": 1,
    "log_interval": 500,
    "dry_run": False,
    "save_model": False,
}

print(config)
main(config)

{'use_hfta': True, 'device': 'cuda', 'batch_size': 64, 'lr': [0.1], 'gamma': 0.7, 'epochs': 4, 'seed': 1, 'log_interval': 500, 'dry_run': False, 'save_model': False}


B=1 lr=[0.1]


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.295558
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.077447
Epoch 1 took 11.392657968999998 s!
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.136890
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.045705
Epoch 2 took 11.238287230000026 s!
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.111560
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.058608
Epoch 3 took 11.479781749999972 s!
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.052281
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.056059
Epoch 4 took 11.362106541000003 s!
Test set: 	Average loss: ['0.0392'], 
 			Accuracy: ['9868/10000(98.68%)']

All jobs Finished, Each epoch took 11.368499729249997 s on average!


### Enable HFTA to train multiple models in the fused form

In [ ]:
# Enable HFTA and fuse 6 models
config = {
    "use_hfta": True,
    "device": "cuda",  # choose from cuda and cpu
    "batch_size": 64,
    "lr": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
    "gamma": 0.7,
    "epochs": 4,
    "seed": 1,
    "log_interval": 500,
    "dry_run": False,
    "save_model": False,
}

print(config)
main(config)

{'use_hfta': True, 'device': 'cuda', 'batch_size': 64, 'lr': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6], 'gamma': 0.7, 'epochs': 4, 'seed': 1, 'log_interval': 500, 'dry_run': False, 'save_model': False}


B=6 lr=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6]


Train Epoch: 1 [0/60000 (0%)]	Loss: 13.828041
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.799471
Epoch 1 took 20.78268953999998 s!
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.668073
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.625135
Epoch 2 took 20.50058612500004 s!
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.187476
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.931624
Epoch 3 took 20.27082759299998 s!
Train Epoch: 4 [0/60000 (0%)]	Loss: 1.565796
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.624585
Epoch 4 took 20.285229027000014 s!
Test set: 	Average loss: ['0.0435', '0.0322', '0.0327', '0.0361', '0.0312', '0.0354'], 
 			Accuracy: ['9854/10000(98.54%)', '9894/10000(98.94%)', '9883/10000(98.83%)', '9879/10000(98.79%)', '9892/10000(98.92%)', '9887/10000(98.87%)']

All jobs Finished, Each epoch took 3.410018879708334 s on average!


From the [Train the model](https://colab.research.google.com/drive/1gSW6PpWAKfHI3GCxOmSrbBS5PFzh7HEl#scrollTo=35jv_fzP-llU) section above, we would know that, if we want to test 6 different learning rates and train each model on a separate GPU, we would need `11.14 * 6 = 66.84` GPU seconds per epoch on average. As we can see, with HFTA, we can reduce the average training time for testing 6 different learning rates to `3.41 * 6 = 20.46` GPU seconds per epoch, thus, improving the overall utilization of the GPU and reducing the overall training time by `66.84 / 20.46 = 3.27x`.

## Conclusion

Based on the time each epoch takes when training the non-HFTA and HFTA version of the same model, we can see that HFTA helps to increase the throughput of the training, especially on a large hardware. Check our [paper](https://arxiv.org/pdf/2102.02344.pdf) for more details.